In [399]:
import numpy as np
import pandas as pd
import random
import csv
import requests
headers = { 
    'accept': "application/json", 
    'apikey': "500bcdd275338226e8f70edc46214483" } 
cities_df = pd.read_csv("./ViolentCrime2017_selectedcities.csv")
cities_df 



,State,City,Population,Violent crime,Property crime,Violent Crime per pop,"Violent crime rate per 100,000 inhabitants",Property crime per pop,"Property crime per 100,000 inhabitants"
0,Virginia,Virginia Beach,"452,797",344,"4,798",0.00,75.97,0.01,1059.64
1,Hawaii,Honolulu,999307,1165,15535,0.00,116.58,0.02,1554.58
2,Kentucky,Lexington,314077,504,5982,0.00,160.47,0.02,1904.63
3,California,Anaheim,349471,620,5151,0.00,177.41,0.01,1473.94
4,Michigan,Detroit,673225,5409,13442,0.01,803.45,0.02,1996.66
5,Maryland,Baltimore,621252,5176,13822,0.01,833.16,0.02,2224.86
6,Tennessee,Memphis,"657,936","5,733","16,833",0.01,871.36,0.03,2558.46
7,Missouri,St. Louis,317095,2781,8522,0.01,877.02,0.03,2687.52


In [437]:

base_url ="https://search.onboard-apis.com/areaapi/v2.0.0/state/lookup"
res = requests.get(base_url,headers=headers).json()
res
st_data =pd.DataFrame(res['response']['result']['package']['item'])
st_data.rename(columns={'name':'State','geo_key':'St_geo_key'}, inplace=True)
statedata = st_data[['State','St_geo_key','abbreviation']]
city_table =pd.merge(cities_df, statedata, on="State", how="left")
city_table 

,State,City,Population,Violent crime,Property crime,Violent Crime per pop,"Violent crime rate per 100,000 inhabitants",Property crime per pop,"Property crime per 100,000 inhabitants",St_geo_key,abbreviation
0,Virginia,Virginia Beach,"452,797",344,"4,798",0.00,75.97,0.01,1059.64,ST51,VA
1,Hawaii,Honolulu,999307,1165,15535,0.00,116.58,0.02,1554.58,ST15,HI
2,Kentucky,Lexington,314077,504,5982,0.00,160.47,0.02,1904.63,ST21,KY
3,California,Anaheim,349471,620,5151,0.00,177.41,0.01,1473.94,ST06,CA
4,Michigan,Detroit,673225,5409,13442,0.01,803.45,0.02,1996.66,ST26,MI
5,Maryland,Baltimore,621252,5176,13822,0.01,833.16,0.02,2224.86,ST24,MD
6,Tennessee,Memphis,"657,936","5,733","16,833",0.01,871.36,0.03,2558.46,ST47,TN
7,Missouri,St. Louis,317095,2781,8522,0.01,877.02,0.03,2687.52,ST29,MO


In [458]:
appended_new = pd.DataFrame([])
for index, row in city_table.iterrows():
    stgc = row['St_geo_key']
    city = row['City']  
    city_url ="https://search.onboard-apis.com/areaapi/v2.0.0/geoid/lookup/?geoId="+stgc+"&GeoType=PL" 
    
    resc = requests.get(city_url,headers=headers).json()
    city_data = pd.DataFrame(resc['response']['result']['package']['item'])
    
    new = city_data.loc[city_data['name'] == city]      
    appended_new = appended_new.append(new)
    
print(appended_new)

    

         area area_unit    geo_key       id            name type
592   497.496   sq. mi.  PL5182000  5182000  Virginia Beach   PL
1482  50.9247   sq. mi.  PL0602000  0602000         Anaheim   PL
691   142.875   sq. mi.  PL2622000  2622000         Detroit   PL
517   92.0506   sq. mi.  PL2404000  2404000       Baltimore   PL
427   326.326   sq. mi.  PL4748000  4748000         Memphis   PL
1027  66.0719   sq. mi.  PL2965000  2965000       St. Louis   PL


In [463]:
city_table1 =pd.merge(city_table, appended_new, left_on=['City'], right_on=['name'])
city_table1 

,State,City,Population,Violent crime,Property crime,Violent Crime per pop,"Violent crime rate per 100,000 inhabitants",Property crime per pop,"Property crime per 100,000 inhabitants",St_geo_key,abbreviation,area,area_unit,geo_key,id,name,type
0,Virginia,Virginia Beach,"452,797",344,"4,798",0.00,75.97,0.01,1059.64,ST51,VA,497.496,sq. mi.,PL5182000,5182000,Virginia Beach,PL
1,California,Anaheim,349471,620,5151,0.00,177.41,0.01,1473.94,ST06,CA,50.9247,sq. mi.,PL0602000,0602000,Anaheim,PL
2,Michigan,Detroit,673225,5409,13442,0.01,803.45,0.02,1996.66,ST26,MI,142.875,sq. mi.,PL2622000,2622000,Detroit,PL
3,Maryland,Baltimore,621252,5176,13822,0.01,833.16,0.02,2224.86,ST24,MD,92.0506,sq. mi.,PL2404000,2404000,Baltimore,PL
4,Tennessee,Memphis,"657,936","5,733","16,833",0.01,871.36,0.03,2558.46,ST47,TN,326.326,sq. mi.,PL4748000,4748000,Memphis,PL
5,Missouri,St. Louis,317095,2781,8522,0.01,877.02,0.03,2687.52,ST29,MO,66.0719,sq. mi.,PL2965000,2965000,St. Louis,PL


In [481]:
appended_main = pd.DataFrame([])

for index, row in city_table1.iterrows():
    cgc = row['geo_key']
  
    base_url="https://search.onboard-apis.com/communityapi/v2.0.0/area/full?AreaId="+cgc
    
    resct = requests.get( base_url,headers=headers).json()
    city_data2 = pd.DataFrame(resct['response']['result']['package']['item'])
    appended_main = appended_main.append(city_data2)
 
    #appended_new = appended_new.append(new)
    

crime = appended_main[['name','latitude','geo_key','longitude','cocrmcyasst','cocrmcytotc']]
print(crime)

             name   latitude    geo_key    longitude cocrmcyasst cocrmcytotc
0  Virginia Beach  36.820655  PL5182000   -76.099708          27          71
0         Anaheim  33.833198  PL0602000  -117.906782          49          55
0         Detroit  42.388561  PL2622000   -83.112219         224         188
0       Baltimore  39.314540  PL2404000   -76.618976         311         272
0         Memphis  35.119649  PL4748000   -89.936550         256         238
0       St. Louis  38.622868  PL2965000   -90.252804         571         475


         area area_unit    geo_key       id            name type
592   497.496   sq. mi.  PL5182000  5182000  Virginia Beach   PL
1482  50.9247   sq. mi.  PL0602000  0602000         Anaheim   PL
691   142.875   sq. mi.  PL2622000  2622000         Detroit   PL
517   92.0506   sq. mi.  PL2404000  2404000       Baltimore   PL
427   326.326   sq. mi.  PL4748000  4748000         Memphis   PL
1027  66.0719   sq. mi.  PL2965000  2965000       St. Louis   PL


In [484]:
schools = pd.DataFrame([])
for index, row in crime.iterrows():
    lat = row['latitude']
    lng = row['longitude']     
    sch_url="https://search.onboard-apis.com/propertyapi/v1.0.0/school/snapshot?latitude="+lat+"&longitude="+lng+"&radius=50&filetypetext=public"
    resch = requests.get(sch_url,headers=headers).json()
print(resch)
    
    #     sch_data = pd.DataFrame(resch['response']['result']['package']['item'])  
#     schools = schools.append(sch_data)
    
# print(schools)

{'status': {'version': '1.0.0', 'code': 0, 'msg': 'SuccessWithResult', 'total': 863, 'page': 1, 'pagesize': 10, 'responseDateTime': None, 'transactionID': None}, 'school': [{'Identifier': {'OBInstID': '5700073700'}, 'School': {'InstitutionName': 'CITY GARDEN MONTESSORI SCHOOL', 'GSTestRating': 4, 'gradelevel1lotext': 'KINDERGARTEN', 'gradelevel1hitext': '8TH GRADE', 'Filetypetext': 'PUBLIC', 'geocodinglatitude': 38.619912, 'geocodinglongitude': -90.255547, 'locationaddress': '1618 TOWER GROVE AVE', 'locationcity': 'SAINT LOUIS', 'stateabbrev': 'MO', 'zip54': '63110-2206', 'distance': 0.25}, 'Vintage': {'onboardDate': '2018-11-28'}}, {'Identifier': {'OBInstID': '04944886'}, 'School': {'InstitutionName': 'ADAMS ELEMENTARY SCHOOL', 'GSTestRating': 0, 'gradelevel1lotext': 'PRESCHOOL', 'gradelevel1hitext': '6TH GRADE', 'Filetypetext': 'PUBLIC', 'geocodinglatitude': 38.625087, 'geocodinglongitude': -90.256612, 'locationaddress': '1311 TOWER GROVE AVE', 'locationcity': 'SAINT LOUIS', 'stateab